In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
from functools import partial
from multiprocessing import Pool

from fastdtw import fastdtw, dtw
from sklearn import svm
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

In [4]:
reduced = pd.read_csv("reduced.csv", index_col="date")
df = reduced.apply(np.log).diff()[1:]
print(df.columns)
data = np.array(df.iloc[0:969])
data

Index(['ABBV', 'ABC', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEE', 'AEP',
       ...
       'WYNN', 'XEL', 'XL', 'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YUM',
       'ZION'],
      dtype='object', length=368)


array([[ 0.00620763,  0.0109497 , -0.00973507, ..., -0.01390149,
        -0.00217596, -0.01752509],
       [-0.01361507, -0.00678224, -0.0136801 , ..., -0.023975  ,
        -0.01212667, -0.02197072],
       [ 0.01063102, -0.00663824,  0.0093228 , ...,  0.00366973,
         0.04441981,  0.01383458],
       ...,
       [ 0.00122945, -0.00459085, -0.00654261, ...,  0.0184337 ,
         0.00455296, -0.00263694],
       [-0.00230645,  0.00287175, -0.00863838, ...,  0.00119048,
         0.00046981, -0.0147506 ],
       [ 0.00184559,  0.00526257, -0.00297245, ...,  0.00966954,
         0.01059035,  0.01090259]])

In [5]:
L = len(data)
l = 20
s = 1
m = len(data[0])

X_train = np.stack([data[i:i + l] for i in range(L // 2 - l - s)])
Y_train = np.empty((L//2 - l - s, m))
for i in range(L//2 - l - s):
    for j in range(m):
        Y_train[i][j] = (data[i + l + s - 1][j] > data[i + l - 1][j]).astype(np.float32)
    

X_test = np.stack([data[i:i + l] for i in range(L // 2, L - l - s)])

d = L - l - s - L//2
Y_test = np.empty((d, m))
for i in range(L // 2, L - l - s):
    for j in range(m):
        Y_test[i - L//2][j] = (data[i + l + s - 1][j] > data[i + l - 1][j]).astype(np.float32)
    


In [6]:
n = len(X_train)
m = len(X_test)
print(n, m)
kernel_train = np.empty((n, n))
kernel_test = np.empty((m, n))

463 464


In [6]:
t0 = time.time()
for i in range(n):
    for j in range(i+1):
        if i == j:
            kernel_train[i][j] = 0.0
        else:
            kernel_train[i][j] = kernel_train[j][i] = fastdtw(X_train[i], X_train[j], radius = 5, dist = 2)[0]
print(kernel_train)
print(time.time()-t0)

[[0.         0.74765186 1.74047067 ... 7.63073172 8.2042119  7.75323047]
 [0.74765186 0.         0.83613807 ... 7.23308943 7.66990485 8.11064687]
 [1.74047067 0.83613807 0.         ... 7.53564254 7.21901302 7.55816945]
 ...
 [7.63073172 7.23308943 7.53564254 ... 0.         0.79112539 1.63615381]
 [8.2042119  7.66990485 7.21901302 ... 0.79112539 0.         0.83100516]
 [7.75323047 8.11064687 7.55816945 ... 1.63615381 0.83100516 0.        ]]
1421.9384269714355


In [9]:
t1 = time.time()
for i in range(m):
    for j in range(n):           
        kernel_test[i][j] = fastdtw(X_test[i], X_train[j], radius = 5, dist = 2)[0]
print(kernel_test)
print(time.time() - t1)

[[7.1283913  6.83908225 6.97183667 ... 6.77964161 6.89568713 7.01284376]
 [7.53656432 7.19899904 6.78150189 ... 6.66771849 6.77844691 7.01952817]
 [7.36996059 7.43576345 6.98311125 ... 6.46238026 6.65894849 6.80278411]
 ...
 [6.95691996 6.99071122 7.12642399 ... 6.56663178 6.76524705 6.63909382]
 [7.10741872 6.93493094 6.9287752  ... 6.6965144  6.63221889 6.86079233]
 [7.32895414 7.10966319 6.92849048 ... 6.73669921 6.79599206 6.72172957]]
2522.9781453609467


In [7]:
##OVO POKREĆES!!!
kernel_train = np.load("kernel_train.npy")

In [8]:
##OVO POKREĆES!!!
kernel_test = np.load("kernel_test.npy")

In [10]:
print(kernel_train.shape)
print(kernel_test.shape)
c = 10
gamma = 25
kernel = np.exp(-kernel_train/gamma)
testkernel = np.exp(-kernel_test/gamma)
clf = svm.SVC(kernel='precomputed', C=c)
l = []
t0 = time.time()
for i in range(data.shape[1]):
    if df.columns[i] == "F":
        clf.fit(kernel, Y_train[:, i])
        res = clf.predict(kernel)
        print('dionica: ', df.columns[i])
        print('accuracy train:', np.mean(res == Y_train[:, i]))
        print('AUC train:', roc_auc_score(Y_train[:, i], res))
        res = clf.predict(testkernel)
        print('accuracy test:', np.mean(res == Y_test[:, i]))
        print('AUC test:', roc_auc_score(Y_test[:, i], res))
        
    


(463, 463)
(464, 463)
dionica:  F
accuracy train: 0.755939524838013
AUC train: 0.7541602782244494
accuracy test: 0.6573275862068966
AUC test: 0.6573275862068966


In [15]:
for i in range(data.shape[1]):
    clf.fit(kernel, Y_train[:, i])
    #res = clf.predict(kernel)
    print('dionica: ', df.columns[i])
    #print('accuracy train:', np.mean(res == Y_train[:, i]))
    #print('AUC train:', roc_auc_score(Y_train[:, i], res))
    res = clf.predict(testkernel)
    print('accuracy test:', np.mean(res == Y_test[:, i]))
    print('AUC test:', roc_auc_score(Y_test[:, i], res))

dionica:  ABBV
accuracy test: 0.5711206896551724
AUC test: 0.5719901856911838
dionica:  ABC
accuracy test: 0.6228448275862069
AUC test: 0.6247791724779173
dionica:  ABT
accuracy test: 0.6422413793103449
AUC test: 0.6447850467289719
dionica:  ACN
accuracy test: 0.6702586206896551
AUC test: 0.6699554069119287
dionica:  ADBE
accuracy test: 0.665948275862069
AUC test: 0.6661983886273563
dionica:  ADI
accuracy test: 0.6724137931034483
AUC test: 0.6722962302361444
dionica:  ADP
accuracy test: 0.6637931034482759
AUC test: 0.6636566332218506
dionica:  ADSK
accuracy test: 0.6271551724137931
AUC test: 0.6271832032701598
dionica:  AEE
accuracy test: 0.5969827586206896
AUC test: 0.5966027356526911
dionica:  AEP
accuracy test: 0.6357758620689655
AUC test: 0.6341383994194593
dionica:  AES
accuracy test: 0.6508620689655172
AUC test: 0.6518554324384622
dionica:  AET
accuracy test: 0.6530172413793104
AUC test: 0.6534601014888753
dionica:  AIV
accuracy test: 0.5883620689655172
AUC test: 0.58810359883321

dionica:  EFX
accuracy test: 0.6314655172413793
AUC test: 0.6286389695480605
dionica:  EIX
accuracy test: 0.625
AUC test: 0.6247490146501078
dionica:  EL
accuracy test: 0.6637931034482759
AUC test: 0.6630792393910666
dionica:  EMN
accuracy test: 0.6443965517241379
AUC test: 0.6429395456152429
dionica:  EMR
accuracy test: 0.6400862068965517
AUC test: 0.6400862068965517
dionica:  EOG
accuracy test: 0.6875
AUC test: 0.6877415997621171
dionica:  EQR
accuracy test: 0.6012931034482759
AUC test: 0.6019744180858183
dionica:  EQT
accuracy test: 0.6853448275862069
AUC test: 0.6851882845188284
dionica:  ETFC
accuracy test: 0.6702586206896551
AUC test: 0.6670400238948625
dionica:  ETN
accuracy test: 0.6400862068965517
AUC test: 0.6415743288465828
dionica:  ETR
accuracy test: 0.5797413793103449
AUC test: 0.5801516080486038
dionica:  EW
accuracy test: 0.5969827586206896
AUC test: 0.5969827586206897
dionica:  EXC
accuracy test: 0.6314655172413793
AUC test: 0.6313193244523717
dionica:  EXPD
accuracy t

dionica:  MMM
accuracy test: 0.6357758620689655
AUC test: 0.6360119047619047
dionica:  MNST
accuracy test: 0.6163793103448276
AUC test: 0.6163793103448277
dionica:  MO
accuracy test: 0.5862068965517241
AUC test: 0.5856558900037161
dionica:  MON
accuracy test: 0.5581896551724138
AUC test: 0.5581436402490012
dionica:  MRK
accuracy test: 0.6379310344827587
AUC test: 0.6389323207505025
dionica:  MRO
accuracy test: 0.7004310344827587
AUC test: 0.7004310344827587
dionica:  MS
accuracy test: 0.6487068965517241
AUC test: 0.6491777385487317
dionica:  MSFT
accuracy test: 0.6724137931034483
AUC test: 0.6730980129742189
dionica:  MSI
accuracy test: 0.625
AUC test: 0.6257062146892656
dionica:  MU
accuracy test: 0.6056034482758621
AUC test: 0.6071428571428572
dionica:  MUR
accuracy test: 0.6637931034482759
AUC test: 0.6650312221231044
dionica:  MYL
accuracy test: 0.6400862068965517
AUC test: 0.6415031222123105
dionica:  NBL
accuracy test: 0.646551724137931
AUC test: 0.6467795095246075
dionica:  NDAQ

AUC test: 0.6417033773861969
dionica:  VAR
accuracy test: 0.6745689655172413
AUC test: 0.6741917502787069
dionica:  VFC
accuracy test: 0.6530172413793104
AUC test: 0.653243037363209
dionica:  VIAB
accuracy test: 0.6379310344827587
AUC test: 0.6391635476602797
dionica:  VLO
accuracy test: 0.6185344827586207
AUC test: 0.6182208349590324
dionica:  VMC
accuracy test: 0.6745689655172413
AUC test: 0.6746580434136188
dionica:  VNO
accuracy test: 0.603448275862069
AUC test: 0.6021609538002981
dionica:  VRSN
accuracy test: 0.6637931034482759
AUC test: 0.6638795986622074
dionica:  VTR
accuracy test: 0.5754310344827587
AUC test: 0.5768014876801488
dionica:  VZ
accuracy test: 0.5517241379310345
AUC test: 0.5512191495688373
dionica:  WAT
accuracy test: 0.6314655172413793
AUC test: 0.6319134701159679
dionica:  WDC
accuracy test: 0.6185344827586207
AUC test: 0.619406596679324
dionica:  WEC
accuracy test: 0.6271551724137931
AUC test: 0.6264036588086561
dionica:  WFM
accuracy test: 0.6293103448275862
A

In [42]:
print(np.mean(l))
print(np.max(l))
print(np.min(l))

0.6316763493253373
0.7219827586206896
0.540948275862069
